# day 19

https://adventofcode.com/2021/day/19

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day19.txt')

LOGGER = logging.getLogger('day19')

## part 1

### problem statement:

#### loading data

In [ ]:
import numpy as np

def load_data(fname=FNAME):
    with open(fname) as fp:
        parsed = {}
        scanner_blocks = [line.split('\n')
                          for line in fp.read().strip().split('\n\n')]
        
        for scanner_block in scanner_blocks:
            scanner_number = int(scanner_block[0].split(' ')[2])
            parsed[scanner_number] = np.array([
                [int(_) for _ in line.split(',')]
                for line in scanner_block[1:]
            ])
        return parsed

In [ ]:
test_data = load_data('data/day19-test.txt')
test_data

In [ ]:
test_data_2d = load_data('data/day19-test-2d.txt')
test_data_2d

In [ ]:
pdist(test_data_2d[0], 'cityblock')

In [ ]:
from scipy.spatial.distance import pdist, squareform

def get_pair_distances(d):
    return {k: pdist(v, 'cityblock') for (k, v) in d.items()}

In [ ]:
get_pair_distances(test_data_2d)

In [ ]:
import itertools

def find_overlaps(d, num_for_overlap=12):
    pair_dists = get_pair_distances(d)
    keys = sorted(pair_dists.keys())
    for (k0, k1) in itertools.combinations(keys, 2):
        v0 = pair_dists[k0]
        v1 = pair_dists[k1]
        overlaps = set(v0).intersection(set(v1))
        if len(overlaps) >= (num_for_overlap * (num_for_overlap - 1)) / 2:
            # calculate the actual directions of the overlapping distances
            # determine the transformation to take basis 0 --> basis 1
            s0 = squareform(v0)
            s1 = squareform(v1)
            
            # find a distance such that both s0 and s1 have one and only one record
            # at that distance
            for dist in list(overlaps):
                try:
                    i00, i01 = np.where(s0 == dist)[0]
                    i10, i11 = np.where(s1 == dist)[0]
                except ValueError:
                    continue
            
            rec00 = d[k0][i00]
            rec01 = d[k0][i01]
            delta0 = rec00 - rec01
            
            rec10 = d[k1][i10]
            rec11 = d[k1][i11]
            delta1 = rec10 - rec11
            
            yield k0, k1, overlaps, ((rec00, rec01, delta0), (rec10, rec11, delta1))

In [ ]:
list(find_overlaps(test_data_2d, 3))

In [ ]:
[(k0, k1, recs) for (k0, k1, _, recs) in find_overlaps(test_data, 12)]

In [ ]:
pair_dists = get_pair_distances(test_data)
keys = sorted(pair_dists.keys())
# for (k0, k1) in itertools.combinations(keys, 2):
#     break
k0, k1 = 2, 4
v0 = pair_dists[k0]
v1 = pair_dists[k1]
overlaps = set(v0).intersection(set(v1))

d = list(overlaps)[0]
s0 = squareform(v0)
s1 = squareform(v1)

np.where(s0 == d)
# i0, j0 = np.where(s0 == d)[0]
# i1, j1 = np.where(s1 == d)[0]

# (i0, j0), (i1, j1)

In [ ]:
(v0 == list(overlaps)[0]).argmax()

#### function def

In [ ]:
def q_1(data):
    return False

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    assert q_1(test_data) == True
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
q_1(load_data())

## part 2

### problem statement:

#### function def

In [ ]:
def q_2(data):
    return False

#### tests

In [ ]:
def test_q_2():
    LOGGER.setLevel(logging.DEBUG)
    assert q_2(test_data) == True
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin

In [ ]:
with open('data/day19.txt') as f:
    ll = [x for x in f.read().strip().split('\n\n')]

ll = [[eval("[" + x + "]") for x in l.split("\n")[1:]] for l in ll]

coord_remaps = [(0, 1, 2), (0, 2, 1), (1, 0, 2), (1, 2, 0), (2, 0, 1), (2, 1, 0)]
coord_negations = [(1, 1, 1), (1, 1, -1), (1, -1, 1), (1, -1, -1), (-1, 1, 1), (-1, 1, -1), (-1, -1, 1), (-1, -1, -1)]
def apply(remap, negat, scan):
    ret = []
    for item in scan:
        ret.append([negat[0]*item[remap[0]], negat[1]*item[remap[1]], negat[2]*item[remap[2]]])
    return ret

distances_from_scan_0 = [(0,0,0)]
def find_alignment(scan_a, scan_b):
    in_a = set([tuple(x) for x in scan_a])
    for remap in coord_remaps:
        for negat in coord_negations:
            a = scan_a
            b = apply(remap, negat, scan_b)
            for a_pos in a:
                for b_pos in b:
                    remap_by = [b_pos[0]-a_pos[0], b_pos[1]-a_pos[1], b_pos[2]-a_pos[2]]
                    matches = 0
                    all_remapped = []
                    for other_b in b:
                        remapped_to_a = (other_b[0]-remap_by[0], other_b[1]-remap_by[1], other_b[2]-remap_by[2])
                        if remapped_to_a in in_a:
                            matches += 1
                        all_remapped.append(list(remapped_to_a))
                    if matches >= 12:
                        print("match", remap_by)
                        distances_from_scan_0.append(tuple(remap_by))
                        return (True, all_remapped)
    return (False, None)

good = ll[0]
aligned_indices = set()
aligned_indices.add(0)
aligned = {}
aligned[0] = ll[0]
all_aligned = []
all_aligned += [tuple(x) for x in ll[0]]
noalign = set()
while len(aligned_indices) < len(ll):
    for i in range(len(ll)):
        if i in aligned_indices:
            continue
        for j in aligned_indices:
            print("Checking", i, "against", j)
            if (i,j) in noalign:
                continue
            ok, remap = find_alignment(aligned[j], ll[i])
            if ok:
                aligned_indices.add(i)
                aligned[i] = remap
                all_aligned += [tuple(x) for x in remap]
                break
            noalign.add((i,j))
print(len(set(all_aligned)))

dists = []
for a in distances_from_scan_0:
    for b in distances_from_scan_0:
        dists.append(sum([abs(a[0]-b[0]), abs(a[1]-b[1]), abs(a[2]-b[2])]))
print(max(dists))